In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [4]:
# -------------------------------
# Paths to train and test folders
# -------------------------------
train_dir = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\train"   # <-- change to your path
test_dir  = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\test"    # <-- change to your path


In [5]:
# -------------------------------
# Image Data Generators
# -------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical"
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
# -------------------------------
# Build CNN Model
# -------------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # auto detect number of emotions
])

In [7]:
# -------------------------------
# Compile Model
# -------------------------------
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [8]:
# -------------------------------
# Callbacks
# -------------------------------
checkpoint = ModelCheckpoint(
    "cnn_emotion_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


In [9]:
# -------------------------------
# Train Model
# -------------------------------
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint, early_stop]
)

Epoch 1/20
449/449 [==============================] - ETA: 0s - loss: 1.8116 - accuracy: 0.2488
Epoch 1: val_accuracy improved from -inf to 0.24868, saving model to cnn_emotion_model.h5
449/449 [==============================] - 153s 339ms/step - loss: 1.8116 - accuracy: 0.2488 - val_loss: 1.7906 - val_accuracy: 0.2487
Epoch 2/20
449/449 [==============================] - ETA: 0s - loss: 1.7854 - accuracy: 0.2541
Epoch 2: val_accuracy improved from 0.24868 to 0.29772, saving model to cnn_emotion_model.h5
449/449 [==============================] - 99s 221ms/step - loss: 1.7854 - accuracy: 0.2541 - val_loss: 1.7232 - val_accuracy: 0.2977
Epoch 3/20
449/449 [==============================] - ETA: 0s - loss: 1.7498 - accuracy: 0.2750
Epoch 3: val_accuracy improved from 0.29772 to 0.32182, saving model to cnn_emotion_model.h5
449/449 [==============================] - 126s 280ms/step - loss: 1.7498 - accuracy: 0.2750 - val_loss: 1.6743 - val_accuracy: 0.3218
Epoch 4/20
449/449 [============

In [10]:
# -------------------------------
# Evaluate Model
# -------------------------------
loss, acc = model.evaluate(validation_generator)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


113/113 [==============================] - 6s 54ms/step - loss: 1.1216 - accuracy: 0.5677
✅ Test Accuracy: 56.77%


In [12]:
# Save final model
model.save("cnn_emotion_model.h5")
print("🎉 Model training completed and saved!")

🎉 Model training completed and saved!


In [13]:
model.save_weights("cnn_model.h5")
